# With PyTorch

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

04/21=The following until train_test_split cell
Is testing a new data structure for easier feeding
into any network, while keeping the time-based
TDS spectral data intact. 


In [2]:
#Load and preprocess the data
folder_path = '500_files'
data = {}

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            df1 = pd.read_csv(file_path, header=None, names=['des_flux', 'detrap_en', 'def_conc'], index_col=None, squeeze = True)
            data[filename] = df1
            
# Combine all the data into a single dataframe
df = pd.concat(data.values())

# Drop NaN values
#df = df.dropna()

display(df)

C:\Users\mvigi\AppData\Local\Temp\ipykernel_26152\4007377030.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df1 = pd.read_csv(file_path, header=None, names=['des_flux', 'detrap_en', 'def_conc'], index_col=None, squeeze = True)
C:\Users\mvigi\AppData\Local\Temp\ipykernel_26152\4007377030.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df1 = pd.read_csv(file_path, header=None, names=['des_flux', 'detrap_en', 'def_conc'], index_col=None, squeeze = True)
C:\Users\mvigi\AppData\Local\Temp\ipykernel_26152\4007377030.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df1 = pd.read_csv(file_path, header=None, names=['des_flux', 'detrap_en', 'def_conc'], index_col=None, sq

,des_flux,detrap_en,def_conc
0,1.269745e-10,1.051587,0.011064
1,8.488521e+14,1.185898,0.006066
2,1.239766e+15,1.203310,0.020460
3,1.784721e+15,1.242848,0.005844
4,2.528399e+15,NaN,NaN
...,...,...,...
296,7.160074e+07,NaN,NaN
297,-6.944291e+08,NaN,NaN
298,-5.921441e+08,NaN,NaN
299,-1.240336e+09,NaN,NaN


In [3]:
#Take each data feature and reshape for 
#the first axis to be equal to number of files
#fed in. In this case, 500. This makes feeding into
#train test splits and networks much easier. 

#Maybe turn this into a loop?
des_flux=df['des_flux']
des_flux=des_flux.tolist()
num_files=int(len(des_flux)/301)
des_flux=np.reshape(des_flux, [num_files, 301])
display(des_flux)
print(type(des_flux[0]), des_flux.shape)

detrap_en=df['detrap_en']
detrap_en=detrap_en.dropna()
detrap_en=detrap_en.tolist()
num_files=int(len(detrap_en)/4)
detrap_en=np.reshape(detrap_en, [num_files, 4])
print(detrap_en, detrap_en.shape)

def_conc=df['def_conc']
def_conc=def_conc.dropna()
def_conc=def_conc.tolist()
num_files=int(len(def_conc)/4)
def_conc=np.reshape(def_conc, [num_files, 4])
print(def_conc, def_conc.shape)

array([[ 1.26974494e-10,  8.48852054e+14,  1.23976624e+15, ...,
        -5.06243457e+11,  1.48607468e+12, -2.00064455e+13],
       [ 0.00000000e+00,  4.75905724e+11,  7.63578326e+11, ...,
        -2.54585985e+10, -4.33822251e+10, -6.32853530e+10],
       [ 0.00000000e+00,  4.90315082e+11,  7.87398576e+11, ...,
         3.05676224e+10,  4.94117204e+10, -6.94978103e+10],
       ...,
       [-1.61113135e-09,  5.30994094e+15,  7.24433559e+15, ...,
        -2.83191940e+11, -2.97233202e+11,  3.12348951e+11],
       [ 0.00000000e+00,  2.09801065e+15,  2.98570846e+15, ...,
         5.66148036e+13,  2.69771268e+13,  1.17249939e+13],
       [ 0.00000000e+00,  3.82317650e+15,  5.08373506e+15, ...,
        -5.92144090e+08, -1.24033587e+09,  1.26747148e+10]])

<class 'numpy.ndarray'> (500, 301)
[[1.05158736 1.1858984  1.20331037 1.24284827]
 [1.25596152 1.5426616  1.63994921 1.86117181]
 [1.25822251 1.45758869 1.51721682 1.66222764]
 ...
 [1.01457836 1.53773973 1.54031238 1.78321747]
 [1.03144402 1.35668867 1.72645702 1.99768237]
 [1.00074039 1.27513497 1.35716673 1.38739602]] (500, 4)
[[0.01106389 0.00606641 0.02045982 0.00584418]
 [0.01107136 0.00256641 0.00625459 0.00236498]
 [0.01906223 0.0150854  0.00796364 0.01020889]
 ...
 [0.02252724 0.0228933  0.00363031 0.02132085]
 [0.01466683 0.00754141 0.01496606 0.01868682]
 [0.00514325 0.00146842 0.00609607 0.00689274]] (500, 4)


In [4]:
#Concatenates the output dataset, creating a 2d array
#of (file, array of detrap_en + def_conc)
#Tried a 3d array of (file, value column, # of values)
#but PyTorch and Tensorflow gave size issues

def_param = np.concatenate([detrap_en, def_conc], axis=1)
print(def_param.shape)

(500, 8)


In [5]:
# Split the data into train and test sets
X_train, X_test= train_test_split(des_flux, test_size=0.1, random_state=42)
y_train, y_test = train_test_split(def_param, test_size=0.1, random_state=42)
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(450, 301)

(50, 301)

(450, 8)

(50, 8)

In [6]:
# Normalize the data
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_norm = scaler.fit_transform(X_train.reshape(-1, 1))
X_test_norm = scaler.transform(X_test.reshape(-1, 1))

In [7]:
print(X_train_norm.mean(), X_train_norm.std(), X_train_norm.min(), X_train_norm.max())
print(X_test_norm.mean(), X_test_norm.std(), X_test_norm.min(), X_test_norm.max())

#Save the values for later use? Necessary or no?
X_train_mean, X_train_std = X_train_norm.mean(), X_train_norm.std()
X_test_mean, X_test_std = X_test_norm.mean(), X_test_norm.std()

-1.9913032299338061e-16 1.0 -0.8483295339669781 5.373506322690123
-0.03368863822273773 0.9900699999615048 -0.8474546306252659 3.858394341848127


In [8]:
#Reshape data for model input
#Checked to make sure the time-series nature of
#the X_train, X_test is intact, can double check
#By calling an array and comparing with a .csv file
X_train_transform = np.reshape(X_train_norm, (X_train.shape))
print(X_train_transform.shape)

X_test_transform = np.reshape(X_test_norm, (X_test.shape))
print(X_test_transform.shape)

(450, 301)
(50, 301)


In [9]:
# Convert data to PyTorch tensors
X_train_transform = torch.tensor(X_train_transform, dtype=torch.float32)
X_test_transform = torch.tensor(X_test_transform, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(type(X_train_transform), type(X_train_transform), type(y_train), type(y_test))

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [10]:
print(X_train_transform.shape, X_test_transform.shape)
print(y_train.shape, y_test.shape)

torch.Size([450, 301]) torch.Size([50, 301])
torch.Size([450, 8]) torch.Size([50, 8])


In [11]:
# Create a DataLoader for the training and test sets
train_dataset = TensorDataset(X_train_transform, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_transform, y_test)
test_loader = DataLoader(test_dataset, batch_size=32)

In [12]:
#Getting the same error as Mai, some value is being converted
#from a tensor to tuple and the evaluate methods don't like it
#but the model runs.

# Define the neural network model
class FeedforwardNN(nn.Module):
    def __init__(self):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(301, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 8)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize the model and optimizer
model = FeedforwardNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the neural network on the test data
model.eval()
with torch.no_grad():
    outputs = model(test_dataset[:, [0]])
    loss = criterion(outputs, test_dataset[:, 1:])
    print('Test Loss: {:.4f}'.format(loss.item()))


Epoch [0/100], Loss: 0.1647
Epoch [10/100], Loss: 0.1614
Epoch [20/100], Loss: 0.1477
Epoch [30/100], Loss: 0.2570
Epoch [40/100], Loss: 0.0903
Epoch [50/100], Loss: 0.0279
Epoch [60/100], Loss: 0.0423
Epoch [70/100], Loss: 0.2210
Epoch [80/100], Loss: 0.0087
Epoch [90/100], Loss: 0.0237


TypeError: linear(): argument 'input' (position 1) must be Tensor, not tuple

# TensorFlow

In [13]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
tf.random.set_seed(42)

In [14]:
# Data already scaled and shaped from PyTorch cells above
#Convert data to TensorFlow tensors
X_train_transform = tf.convert_to_tensor(X_train_transform, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_transform = tf.convert_to_tensor(X_test_transform, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [15]:
#TensorFlow runs as well now, just not well.
#Now we can optimize. 

# Define the model architecture
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=64, activation='relu', 
                                    kernel_initializer=tf.keras.initializers.GlorotUniform()))
model.add(tf.keras.layers.Dense(units=32, activation='relu',
                                    kernel_initializer=tf.keras.initializers.GlorotUniform()))
model.add(tf.keras.layers.Dense(units=8, activation='softmax',
                                    kernel_initializer=tf.keras.initializers.GlorotUniform()))

# Compile the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1), loss='mse')

# Train the model
history = model.fit(X_train_transform, y_train, epochs=10, validation_split=0.1)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test_transform, y_test)
print(f'Test loss: {test_loss:.4f}')


Epoch 1/10
13/13 [==============================] - 1s 18ms/step - loss: 0.9176 - val_loss: 0.8915
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 5/10
13/13 [==============================] - 0s 7ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 6/10
13/13 [==============================] - 0s 7ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 8/10
13/13 [==============================] - 0s 4ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 9/10
13/13 [==============================] - 0s 9ms/step - loss: 0.9073 - val_loss: 0.8915
Epoch 10/10
2/2 [==============================] - 0s 6ms/step - loss: 0.8763
Test loss: 0.8763
